In [2]:
import ROOT
print(ROOT.__version__)

# why it matters? because Note: RooMinuit is replaced by RooMinimizer starting from ROOT v6.30

6.34.00


In [29]:
x = ROOT.RooRealVar("x", "x", -20, 20)
mean = ROOT.RooRealVar("mean", "mean", 0)
s1 = ROOT.RooRealVar("s1", "sigma", 3.3)  # taken my matriculation number randomly as 3
s2 = ROOT.RooRealVar("s2", "sigma", 4, 3, 6)
f = ROOT.RooRealVar("f", "coefficient", 0.5, 0.3, 1)

In [30]:
# Generate a dataset
# Composite Model: The sum of two Gaussians: f⋅gaus1(..)+(1−f)⋅gaus2(..) 

# Gaussian Model 1
gauss1 = ROOT.RooGaussian("gaus1", "gaussian dist1", x, mean, s1)

# Gaussian Model 2
gauss2 = ROOT.RooGaussian("gaus2", "gaussian dist2", x, mean, s2)

model = ROOT.RooAddPdf("m", "m", ROOT.RooArgList(gauss1, gauss2), ROOT.RooArgList(f))


# Generate dataset
data = model.generate({x}, 1000)

[#0] WARNING:InputArguments -- The parameter 's1' with range [-inf, inf] of the RooGaussian 'gaus1' exceeds the safe range of (0, inf). Advise to limit its range.


Create workspace and save the model

In [34]:
w = ROOT.RooWorkspace("w", "workspace")

# Import model and data into the workshape
w.Import(model)
w.Import(data)

# save the data to file
w.writeToFile("/mnt/d/OneDrive - Alma Mater Studiorum Università di Bologna/ROOT/Lecture2/output/workshape_model1.root")

False

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooAddPdf::m
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooGaussian::gaus1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::x
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::mean
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::s1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::f
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooGaussian::gaus2
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::s2
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing dataset mData


In [35]:
# Construct unbinned likelihood -log(L) of PDF with given dataset
neg_log_likelihood = model.createNLL(data)

# Pass the likelihood directly to a RooMinuit object
minute_obj = ROOT.RooMinimizer(neg_log_likelihood)

[#1] INFO:Fitting -- RooAbsPdf::fitTo(m) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_m_mData) Summation contains a RooNLLVar, using its error level


HESSE Error Calculation 

In [40]:
# Disable verbose mode
minute_obj.setVerbose(False)

# Running HESSE to calculate errors d2L/dp2
minute_obj.hesse()

# Display the values of parameters
f.Print()
mean.Print()
s1.Print()
s2.Print()

0

Info in <Minuit2>: Minuit2Minimizer::Hesse Using max-calls 1000
Info in <Minuit2>: Minuit2Minimizer::Hesse Hesse is valid - matrix is accurate


MINOS Error Calculation for “s2”

In [42]:
# Call MINOS for the parameter “s2”
minute_obj.minos({s2})

# Display the values of parameters
f.Print()
mean.Print()
s1.Print()
s2.Print()

******************************************************************************************************
Minuit2Minimizer::GetMinosError - Run MINOS LOWER error for parameter #1 : s2 using max-calls 1000, tolerance 1
******************************************************************************************************
Minuit2Minimizer::GetMinosError - Run MINOS UPPER error for parameter #1 : s2 using max-calls 1000, tolerance 1
Minos: Lower error for parameter s2  :  -0.1095
Minos: Upper error for parameter s2  :  0.6407
RooRealVar::f = 0.3 +/- 0.35  L(0.3 - 1) 
RooRealVar::mean = 0 C  L(-INF - +INF) 
RooRealVar::s1 = 3.3 C  L(-INF - +INF) 
RooRealVar::s2 = 3.652 +/- (-0.1095,0.6407)  L(3 - 6) 


Info in <Minuit2>: MnMinos Determination of lower Minos error for parameter 1
Info in <Minuit2>: MnFunctionCross Run Migrad with fixed parameters:
  Pos 1: s2 = 3.37403
Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       2689.537509 Edm =      0.6231005705 NCalls =      3
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 2689.537509
  Edm           : 0.6231005705
  Internal parameters:	[     -1.081240494]	
  Internal gradient  :	[     0.4483037391]	
  Internal covariance matrix:
[[      12.401477]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.0005 with call limit = 1000
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       2689.537509 Edm =      0.6231005705 NCalls =      3
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =       2689.431956 Edm =     0.03131254061 NCalls =      7
Info in <Minuit2>: VariableMetricBuilder    2 - FCN =  

Save Results

In [47]:
# Save a snapshot of the fit result
r = minute_obj.save()

# Print the fit result snapshot
r.Print("v")



# Set the color palette for the plot
ROOT.gStyle.SetPalette(1)

# Draw the correlation matrix of the fit result
r.correlationHist().Draw("colz")


### EXTRAS 
# Create a canvas to draw the correlation matrix
canvas = ROOT.TCanvas("canvas", "Correlation Matrix", 800, 600)

# Draw the correlation matrix of the fit result on the canvas and save it
r.correlationHist().Draw("colz")
canvas.SaveAs("correlation_matrix.png")

# Save the correlation matrix to a ROOT file
output_file = ROOT.TFile("correlation_matrix.root", "RECREATE")
r.correlationHist().Write("correlationMatrix")
output_file.Close()


  RooFitResult: minimized FCN value: 2686, estimated distance to minimum: 0.0002206
                covariance matrix quality: Full, accurate covariance matrix
                Status : MIGRAD=0 MIGRAD=0 HESSE=0 MINOS=0 

    Constant Parameter    Value     
  --------------------  ------------
                  mean    0.0000e+00
                    s1    3.3000e+00

    Floating Parameter  InitialValue    FinalValue (+HiError,-LoError)    GblCorr.
  --------------------  ------------  ----------------------------------  --------
                     f    5.0000e-01    3.0005e-01         +/-  3.50e-01  <none>
                    s2    4.0000e+00    3.6523e+00 (+6.41e-01,-1.10e-01)  <none>



Warning in <TROOT::Append>: Replacing existing TH1: correlation_matrix (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: correlation_matrix (Potential memory leak).
Info in <TCanvas::Print>: png file correlation_matrix.png has been created


Contour plot

In [ ]:
counter_plot = minute_obj.counnter(f, s2, 68, 95.45, 99.73)
minute_obj.ErrorDef(0.5)  ## 0.5 for negative log likelihood (given in documentation)

In [1]:
x = ROOT.RooRealVal("x", "x", 1, 0, 10)
xframe = x.frame()
model.plotOn(xframe)
xframe.draw()



NameError: name 'ROOT' is not defined

[3] Hands-on: Composite Model; Working with Likelihoods  (ADVANCED) 


In [ ]:
# define the observable neutrino energy
energy = ROOT.RooRealVar("E", "Reconstructed neutrino energy", 0.5, 14, "GeV")

# load the data
